In [2]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np

# Plotting library
import matplotlib.pyplot as plt

import pandas as pd

# tells matplotlib to embed plots within the notebook
%matplotlib inline

# Numerical Data only test

### Import the training dataset and Feature engineering

In [35]:
# import training CSV
fname = './data/train.csv'
# use pandas to read the CSV properly of all different types
data_pd = pd.read_csv(fname)
# convert to a numpy array
data = np.array(data_pd)
# remove categorical data
done = False
i = 0
while(not done):

    if i != data.shape[1]:
        #print(type(data[:,i][0]))
        #check to see if column contains a string
        contains_string = False
        for val in data[:,i]:
            if type(val) == str:
                contains_string = True
        if contains_string:
            data = np.delete(data, i, 1)
            continue
        else:
            i += 1
    else:
        done = True
# remove outliers with standard deviation TODO()

# replace example # column with 1's intercept column
data[:,0] = np.ones(data.shape[0])
# take the actual prices and add them to results y
y = data[:, data.shape[1] - 1]
# remove prices column from data
data = np.delete(data, data.shape[1] - 1, 1)
data = data.astype(float)
print("data shape", data.shape)
data

data shape (1460, 37)


array([[1.000e+00, 6.000e+01, 6.500e+01, ..., 0.000e+00, 2.000e+00,
        2.008e+03],
       [1.000e+00, 2.000e+01, 8.000e+01, ..., 0.000e+00, 5.000e+00,
        2.007e+03],
       [1.000e+00, 6.000e+01, 6.800e+01, ..., 0.000e+00, 9.000e+00,
        2.008e+03],
       ...,
       [1.000e+00, 7.000e+01, 6.600e+01, ..., 2.500e+03, 5.000e+00,
        2.010e+03],
       [1.000e+00, 2.000e+01, 6.800e+01, ..., 0.000e+00, 4.000e+00,
        2.010e+03],
       [1.000e+00, 2.000e+01, 7.500e+01, ..., 0.000e+00, 6.000e+00,
        2.008e+03]])

### Normal Equation and Polynomial Regression Functions

### 3.2 Normal equation

We have learned that the closed-form solution to linear regression is

$$ \theta = \left( X^T X\right)^{-1} X^T\vec{y}$$

Using this formula does not require any feature scaling, and you will get an exact solution in one calculation: there is no “loop until convergence” like in gradient descent. 

Hint: Before implement normal_equation function, following example shows how to convert array data to a matrix with a shape of $m \times 1$, and generate polynomial features matrix $[1, X, X^2, X^3, \cdots]$. 

In [31]:
# [CHECKPOINT 2][10 points]
def normal_equation(X, Y, n):
    """
    Computes the closed-form solution to linear regression using the normal equations.
    
    Parameters
    ----------
    X : array_like
        The dataset of shape (m, ).
    
    Y : array_like
        The value at each data point. A vector of shape (m, ).
        
    n : the order of polynomial regression model
        Remember the number of features will be n+1.
    
    Returns
    -------
    theta : array_like
        Estimated polynomial regression parameters. A vector of shape (n+1, ).
    
    Instructions
    ------------
    Complete the code to compute the closed form solution to linear
    regression and put the result in theta.
    
    Hint
    ----
    Look up the function `np.linalg.pinv` for computing matrix inverse.
    """
    #print("Normal equation start")
    #print("n: ", n)
    m = X.size
    print("m: ", m)
    theta = np.zeros(n+1)
    #print("theta: ", theta)
    
    # ===================== YOUR CODE HERE ============================
    X_t = np.transpose(X)
    
    '''X_col = X[:m].reshape(m,1)
    #print("X_col: \n", X_col)
    #print("X_col size:", X_col.shape)
    X_poly = np.power(X_col, np.arange(n+1))
    #print("X poly powered: \n", X_poly)
    #ones = np.ones(m).reshape(m, 1)
    #X_poly = np.append(ones, X_poly, axis = 1)
    #print("X poly ones appended: \n", X_poly)
    #transposing
    X_t = np.transpose(X_poly)
    #print("X_poly transpose: \n", X_t)
    #finding dot product
    X_poly = X_t.dot(X_poly)
    #print("Multiplied by transpose: \n", X_poly)
    #taking inverse
    X_poly = np.linalg.pinv(X_poly)
    #print("inverse: \n", X_poly)
    #dot product with transpose again
    X_poly = X_poly.dot(X_t)
    #print("X poly by transpose: \n", X_poly)
    #print("X shape: ", X_poly.shape, "Y shape: ", Y.shape)
    X_poly = X_poly.dot(Y)
    #print("X poly by y: \n", X_poly)
    theta = X_poly    '''
    # =================================================================
    return theta.flatten()

In [32]:
# [CHECKPOINT 3][5 points]
def polynomial_deploy(X, theta):
    """
    Computes the polynomial regression prediction for data X.
    
    Parameters
    ----------
    X : array_like
        The input data. A vector of shape (m, ).
    
    theta : array_like
        Polynomial regression parameters. A vector of shape (n+1, ).
    
    Returns
    -------
    Y : array_like
        Polynomial prediction. A vector of shape (m, ).
    
    """
    #print("Begin polynomial deploy")
    m = X.size
    n = theta.size - 1
    Y = np.array([])
    
    # ===================== YOUR CODE HERE ============================
    #print("theta: ", theta)
    
    #print("X size", X.shape)
    X_col = X_data[:m].reshape(m,1)
    
    for i in range(m): #for each data point
        prediction = 0
        for j in range(n+1): #for each degree
            prediction += (theta[j] * X[i] ** j) 
            #print("theta: ", theta[j], "X val: ", X[i])
        #print("prediction for X: ", X[i], " is: ", prediction)
        Y = np.append(Y, prediction)
            
    #print("X poly size: ", X_poly.shape)
    
    # ===================== YOUR CODE HERE ============================
    return Y.flatten()

In [33]:
#MSE helper func
def MSE_func(m, hypothesis, actual):
    MSE = 0
    for i in range(m):
            MSE += (hypothesis[i] - actual[i]) ** 2
    return MSE

### Polynomial Function Predictions and MSE

In [34]:
n = 1
X_data = data
theta = normal_equation(X_data, y, n)
print("theta x set: ", theta)
prediction = polynomial_deploy(X_data, theta)
MSE = MSE_func(X_data.size, prediction, y)
plt.figure()
plt.title('m=%d, n=%d, MSE=%.8f' % (m, n, MSE))
plt.plot(X_data, y, 'b+')
Y_predict = polynomial_deploy(X_data, theta)
plt.plot(X_data, Y_predict, 'r-')
print("plot complete")

m:  54020
X_col: 
 [[1.000e+00]
 [6.000e+01]
 [6.500e+01]
 ...
 [0.000e+00]
 [6.000e+00]
 [2.008e+03]]
X_col size: (54020, 1)


ValueError: shapes (2,54020) and (1460,) not aligned: 54020 (dim 1) != 1460 (dim 0)